In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType

In [0]:
%fs ls dbfs:/FileStore/tables/

path,name,size,modificationTime
dbfs:/FileStore/tables/Address.xlsx,Address.xlsx,151315,1706080470000
dbfs:/FileStore/tables/Detail-1.csv,Detail-1.csv,208476,1706073797000
dbfs:/FileStore/tables/Detail-2.csv,Detail-2.csv,208476,1706074158000
dbfs:/FileStore/tables/Detail-3.csv,Detail-3.csv,208476,1706074878000
dbfs:/FileStore/tables/Detail.csv,Detail.csv,208476,1706073648000
dbfs:/FileStore/tables/contactinfo.txt,contactinfo.txt,49969,1706075815000
dbfs:/FileStore/tables/csv/,csv/,0,0
dbfs:/FileStore/tables/header-1.json,header-1.json,115560,1706078871000
dbfs:/FileStore/tables/header-2.json,header-2.json,115560,1706081187000
dbfs:/FileStore/tables/header.json,header.json,115560,1706076762000


In [0]:
%fs ls dbfs:/FileStore/tables/csv/


path,name,size,modificationTime
dbfs:/FileStore/tables/csv/batch.csv,batch.csv,151,1707460773000


#Task 1


load both csv and json. instead of dropping duplicates, create a boolean column is_duplicate and set False to only one row and true to rest of them.

In [0]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("dob", DateType(), True),
    StructField("age", IntegerType(), True),
    StructField("salary", IntegerType(), True),
    StructField("department", StringType(), True)])

In [0]:
df_csv=spark.read.format("csv").option("header",True).schema(schema).load("dbfs:/FileStore/tables/csv/batch.csv")
df_csv.show()
df_csv.printSchema()

+---+-----+----------+----+------+----------+
| id| name|       dob| age|salary|department|
+---+-----+----------+----+------+----------+
|  1| John|1992-05-12|  30| 70000|        IT|
|  2|Alice|1997-02-28|  25| 60000|        HR|
|  3|  Bob|      null|null| 80000|        IT|
|  4|Emily|1994-11-22|  28| 65000|   Finance|
+---+-----+----------+----+------+----------+

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: string (nullable = true)



In [0]:
df_json=spark.read.format("json").option("header",True).schema(schema).load("dbfs:/FileStore/tables/json/")
df_json.show()
df_json.printSchema()

+---+------+----------+----+------+----------+
| id|  name|       dob| age|salary|department|
+---+------+----------+----+------+----------+
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|
|  2| Alice|1997-02-28|  25| 90000|   Finance|
|  4| Emily|1994-11-22|  28| 70000|   Finance|
|  9| James|1983-10-14|  39| 87000|        IT|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|
+---+------+----------+----+------+----------+

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department:

In [0]:
df=df_csv.union(df_json)

In [0]:
df.show()
print(df.count())

+---+------+----------+----+------+----------+
| id|  name|       dob| age|salary|department|
+---+------+----------+----+------+----------+
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|
|  2| Alice|1997-02-28|  25| 90000|   Finance|
|  4| Emily|1994-11-22|  28| 70000|   Finance|
|  9| James|1983-10-14|  39| 87000|        IT|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|
+---+------+----------+----+------+----------+

17


In [0]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: string (nullable = true)



In [0]:
df1=df.dropDuplicates()
print(df1.count())

12


In [0]:
df1.show()

+---+------+----------+----+------+----------+
| id|  name|       dob| age|salary|department|
+---+------+----------+----+------+----------+
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|
|  2| Alice|1997-02-28|  25| 90000|   Finance|
|  4| Emily|1994-11-22|  28| 70000|   Finance|
|  9| James|1983-10-14|  39| 87000|        IT|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|
+---+------+----------+----+------+----------+



In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as f

In [0]:
df2 = df.withColumn("is_duplicate", (f.row_number().over(Window.partitionBy(df.columns).orderBy("id")) > 1))

In [0]:
df2.show()
df2.printSchema()

+---+------+----------+----+------+----------+------------+
| id|  name|       dob| age|salary|department|is_duplicate|
+---+------+----------+----+------+----------+------------+
|  1|  John|1992-05-12|  30| 70000|        IT|       false|
|  1|  John|1992-05-12|  30| 70000|        IT|        true|
|  1|  John|1992-05-12|  30| 70000|        IT|        true|
|  2| Alice|1997-02-28|  25| 60000|        HR|       false|
|  2| Alice|1997-02-28|  25| 60000|        HR|        true|
|  2| Alice|1997-02-28|  25| 90000|   Finance|       false|
|  3|   Bob|      null|null| 80000|        IT|       false|
|  3|   Bob|      null|null| 80000|        IT|        true|
|  4| Emily|1994-11-22|  28| 65000|   Finance|       false|
|  4| Emily|1994-11-22|  28| 65000|   Finance|        true|
|  4| Emily|1994-11-22|  28| 70000|   Finance|       false|
|  5| David|1981-12-18|  41| 90000|        HR|       false|
|  6| Susan|1989-07-05|  33| 75000|   Finance|       false|
|  7|  Mike|1976-03-15|  46| 95000|     

#Task 2

Calculate mean salary and check if it is greater or equal to the salary of employees in each department.
 
Calculate mean salary and check if it is greater or equal to the salary of all employees.

In [0]:
mean_obj = Window.partitionBy(df.department)

In [0]:
df3 = df.withColumn("is_greter_Than_Deptmean",f.avg("salary").over(mean_obj)<=df.salary)

In [0]:
df3.show()

+---+------+----------+----+------+----------+-----------------------+
| id|  name|       dob| age|salary|department|is_greter_Than_Deptmean|
+---+------+----------+----+------+----------+-----------------------+
|  4| Emily|1994-11-22|  28| 65000|   Finance|                  false|
|  4| Emily|1994-11-22|  28| 65000|   Finance|                  false|
|  6| Susan|1989-07-05|  33| 75000|   Finance|                   true|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|                  false|
|  2| Alice|1997-02-28|  25| 90000|   Finance|                   true|
|  4| Emily|1994-11-22|  28| 70000|   Finance|                  false|
|  2| Alice|1997-02-28|  25| 60000|        HR|                  false|
|  2| Alice|1997-02-28|  25| 60000|        HR|                  false|
|  5| David|1981-12-18|  41| 90000|        HR|                   true|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|                  false|
|  1|  John|1992-05-12|  30| 70000|        IT|                  false|
|  3| 

In [0]:
mean_obj1 = df.select(f.avg("salary")).first()[0]

In [0]:
print(mean_obj1)

73352.94117647059


In [0]:
df4 = df.withColumn("is_greter_Than_Allmean",mean_obj1 <= df.salary)

In [0]:
df4.show()

+---+------+----------+----+------+----------+----------------------+
| id|  name|       dob| age|salary|department|is_greter_Than_Allmean|
+---+------+----------+----+------+----------+----------------------+
|  1|  John|1992-05-12|  30| 70000|        IT|                 false|
|  2| Alice|1997-02-28|  25| 60000|        HR|                 false|
|  3|   Bob|      null|null| 80000|        IT|                  true|
|  4| Emily|1994-11-22|  28| 65000|   Finance|                 false|
|  1|  John|1992-05-12|  30| 70000|        IT|                 false|
|  2| Alice|1997-02-28|  25| 60000|        HR|                 false|
|  3|   Bob|      null|null| 80000|        IT|                  true|
|  4| Emily|1994-11-22|  28| 65000|   Finance|                 false|
|  5| David|1981-12-18|  41| 90000|        HR|                  true|
|  6| Susan|1989-07-05|  33| 75000|   Finance|                  true|
|  7|  Mike|1976-03-15|  46| 95000|        IT|                  true|
| 10|Sophie|1992-06-

In [0]:
w3=Window.partitionBy()

In [0]:
df5=df.withColumn("sal_w3",f.avg("salary").over(w3)).show()

+---+------+----------+----+------+----------+-----------------+
| id|  name|       dob| age|salary|department|           sal_w3|
+---+------+----------+----+------+----------+-----------------+
|  1|  John|1992-05-12|  30| 70000|        IT|73352.94117647059|
|  2| Alice|1997-02-28|  25| 60000|        HR|73352.94117647059|
|  3|   Bob|      null|null| 80000|        IT|73352.94117647059|
|  4| Emily|1994-11-22|  28| 65000|   Finance|73352.94117647059|
|  1|  John|1992-05-12|  30| 70000|        IT|73352.94117647059|
|  2| Alice|1997-02-28|  25| 60000|        HR|73352.94117647059|
|  3|   Bob|      null|null| 80000|        IT|73352.94117647059|
|  4| Emily|1994-11-22|  28| 65000|   Finance|73352.94117647059|
|  5| David|1981-12-18|  41| 90000|        HR|73352.94117647059|
|  6| Susan|1989-07-05|  33| 75000|   Finance|73352.94117647059|
|  7|  Mike|1976-03-15|  46| 95000|        IT|73352.94117647059|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|73352.94117647059|
|  2| Alice|1997-02-28|  